<a href="https://colab.research.google.com/github/kevintsi/deep_learning_course/blob/main/project_dl_covid_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Projet Deep Learning : Détection de COVID

In [68]:
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

OSError: ignored

In [ ]:
from fastbook import *
from fastai.vision.widgets import *

In [ ]:
Ntrain = 400
Nval = 200

covid_sample_path = f"drive/MyDrive/covid_detection_sample_{Ntrain}_Ntrain_{Nval}_Nval"

In [ ]:
state_types = ["COVID", "Normal", "Opacite_Pulmonaire", "Pneumonie_Virale"] 

In [ ]:
fns = get_image_files(covid_sample_path+"/train")
fns

In [ ]:
failed = verify_images(fns)
failed

In [ ]:
failed.map(Path.unlink);

In [ ]:
states = DataBlock(
    blocks=(ImageBlock, CategoryBlock), 
    get_items=get_image_files, 
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=Resize(128))

In [ ]:
dls = states.dataloaders(covid_sample_path)

In [ ]:
dls.valid.show_batch(max_n=10, nrows=2)

In [ ]:
states = states.new(item_tfms=Resize(128, ResizeMethod.Squish))
dls = states.dataloaders(covid_sample_path)
dls.valid.show_batch(max_n=10, nrows=2)

In [ ]:
states = states.new(item_tfms=Resize(128, ResizeMethod.Pad, pad_mode="zeros"))
dls = states.dataloaders(covid_sample_path)
dls.valid.show_batch(max_n=10, nrows=2)

In [ ]:
states = states.new(item_tfms=RandomResizedCrop(175, min_scale=0.8))
dls = states.dataloaders(covid_sample_path)
dls.valid.show_batch(max_n=10, nrows=2)

In [ ]:
states = states.new(item_tfms=Resize(128), batch_tfms=aug_transforms(mult=2))
dls = states.dataloaders(covid_sample_path)
dls.valid.show_batch(max_n=10, nrows=2, unique=True)

In [ ]:
states = states.new(item_tfms=RandomResizedCrop(275, min_scale=0.6), batch_tfms=aug_transforms())
dls = states.dataloaders(covid_sample_path)

In [ ]:
learn = cnn_learner(dls, resnet18, metrics=error_rate)
learn.fine_tune(4)

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix()

In [ ]:
interp.plot_top_losses(5, nrows=1)

In [ ]:
cleaner = ImageClassifierCleaner(learn)
cleaner

In [ ]:
for idx in cleaner.delete(): cleaner.fns[idx].unlink()

In [ ]:
for idx,cat in cleaner.change(): shutil.move(str(cleaner.fns[idx]), path/cat)

In [ ]:
learn.export()

In [ ]:
path = Path(covid_sample_path)
path.ls(file_exts='.pkl')